# 최적화 2조
## 정윤서, 연현중, 정승우, 진경배, 최원준

In [10]:
!pip install haversine
!pip install gurobipy
!pip install numpy<2
!pip install ortools
!pip install pandas


[notice] A new release of pip is available: 23.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
zsh:1: no such file or directory: 2

[notice] A new release of pip is available: 23.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [11]:
import csv
import random
from haversine import haversine, Unit
import gurobipy as gp
from gurobipy import Model, GRB

## 창고 및 직영점 위치 선정
### 랜덤으로 창고 후보지 선정 후 창고와 직영점 선택

In [20]:
def read_csv_with_encoding(file_path):
    encodings = ['utf-8', 'ISO-8859-1', 'cp1252']
    for enc in encodings:
        try:
            with open(file_path, newline='', encoding=enc) as csvfile:
                reader = csv.reader(csvfile)
                data = [row for row in reader]
            return data
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to read the file {file_path} with tried encodings.")

# 데이터 저장할 리스트 초기화
cafe_locations = []
sales = []

# CSV 파일에서 좌표 및 매출 읽기
cafe_data = read_csv_with_encoding('각 구청별 좌표.CSV')
for row in cafe_data[0:]:  # Skip header row
    cafe_locations.append((float(row[0]), float(row[1])))
    sales.append(int(row[2]))

# 창고 후보 위치 읽기
warehouse_locations = []
warehouse_data = read_csv_with_encoding('candidate_storages.csv')
for row in warehouse_data[1:]:  # Skip header row
    warehouse_locations.append((float(row[2]), float(row[3])))

# 좌표 개수
num_cafe_coords = len(cafe_locations)
num_warehouse_coords = len(warehouse_locations)

# 거리 행렬 초기화
cafe_distances = [[0.0 for _ in range(num_cafe_coords)] for _ in range(num_cafe_coords)]
warehouse_distances = [[0.0 for _ in range(num_warehouse_coords)] for _ in range(num_cafe_coords)]

# 각 좌표 쌍 간의 거리 계산
for i, loc1 in enumerate(cafe_locations):
    for j, loc2 in enumerate(cafe_locations):
        if i != j:  # 자기 자신과의 거리는 0으로 유지
            cafe_distances[i][j] = haversine(loc1, loc2, unit='km')
        else:
            cafe_distances[i][j] = 1000000000000  # 대규모 상수로 설정하여 자기 자신과의 경로 선택 방지

for i, loc1 in enumerate(cafe_locations):
    for j, loc2 in enumerate(warehouse_locations):
        warehouse_distances[i][j] = haversine(loc1, loc2, unit='km')


# 랜덤으로 창고 후보 위치 50개 선택
random.seed(42)
selected_warehouse_indices = random.sample(range(num_warehouse_coords), 50)

# Set
FF = selected_warehouse_indices  # 창고 후보 (랜덤으로 선택된 50개)
CC = list(range(num_cafe_coords))  # 모든 카페 노드

# Parameter
d_fc = {(i, j): cafe_distances[i][j] for i in CC for j in CC}  # 거리 직영점끼리
c_fc = 0.13  # 운임비 창고-직영점
M = 1000000000

# Model
m = gp.Model("coffee_fee")

# Decision Variables
C = m.addVars(CC, CC, vtype=GRB.BINARY, name='C')  # 직영점 끼리의 동선 여부
F = m.addVars(FF, vtype=GRB.BINARY, name='F')      # 평창-창고 여부
Ck = m.addVars(CC, vtype=GRB.BINARY, name='Ck')    # 직영점 여부
V = m.addVars(FF, CC, vtype=GRB.BINARY, name='V')  # 창고-첫 번째 직영점 동선 여부

# Objective function to maximize sales while minimizing costs
alpha = 1e6  # Large constant to prioritize sales

obj = (
    4 * c_fc * gp.quicksum(d_fc[i, j] * C[i, j] for i in CC for j in CC)  # Distribution cost between stores
    - alpha * gp.quicksum(sales[j] * Ck[j] for j in CC)  # Expected sales
)
m.setObjective(obj, GRB.MINIMIZE)

# Constraints
m.addConstr(gp.quicksum(F[i] for i in FF) == 1, "one_warehouse")  # Only one warehouse

# Ensure each store is either selected or not
m.addConstrs(
    (gp.quicksum(C[i, j] for j in CC) <= Ck[i] for i in CC), name="path_constraint"
)
m.addConstrs(
    (gp.quicksum(C[i, j] for i in CC) <= Ck[j] for j in CC), name="path_constraint"
)

# Ensure adjacency constraint for stores
adjacency_constraints = [
    (0, 1), (0, 3), (0, 4), (1, 2), (1, 4), (2, 4), (2, 5), (2, 10), (2, 4), (3, 4),
    (3, 6), (3, 7), (3, 8), (4, 9), (5, 10), (6, 7), (6, 11), (7, 11), (7, 12),
    (8, 9), (8, 12), (8, 13), (8, 17), (8, 18), (9, 10), (9, 13), (10, 13), (11, 12),
    (11, 15), (12, 17), (13, 17), (13, 18), (14, 15), (14, 19), (14, 20), (15, 16),
    (15, 20), (15, 21), (16, 17), (16, 21), (17, 18), (17, 22), (18, 22), (18, 23),
    (19, 20), (20, 21), (21, 22), (22, 23), (22, 24), (23, 24)
]

m.addConstrs(
    (Ck[i] + Ck[j] <= 1 for i, j in adjacency_constraints if i in CC and j in CC), name="adjacency_constraint"
)

# Constraint on total sales
m.addConstr(
    gp.quicksum(0.044 * sales[j] * Ck[j] for j in CC) <= 1388431.8, name="sales_constraint"
)

# Solve TSP including the warehouse
for i in FF:
    for j in CC:
        if j != i:
            m.addConstr(V[i, j] <= F[i], name=f"first_delivery_{i}_{j}")

m.optimize()

# Extract selected cafes
selected_cafes = []
selected_warehouse = None
if m.status == GRB.Status.OPTIMAL:
    print('Optimal objective: %g' % m.objVal)
    for v in m.getVars():
        if v.x > 0:
            print('%s: %g' % (v.varName, v.x))
    
    # Extract selected cafes and warehouse
    selected_cafes = [j for j in CC if Ck[j].x > 0.5]
    for i in FF:
        if F[i].x > 0.5:
            selected_warehouse = warehouse_locations[i]
            break
else:
    print('No optimal solution found')

# Print selected cafes and warehouse
print("Selected Cafes:", selected_cafes)
print("Selected Warehouse:", selected_warehouse)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1345 rows, 1950 columns and 3961 nonzeros
Model fingerprint: 0x1efb5546
Variable types: 0 continuous, 1950 integer (1950 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e-01, 5e+11]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -8.69800e+09
Presolve removed 1345 rows and 1950 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -1.9048e+10 -8.698e+09 
No other solutions better than -1.9048e+10

Optimal solution found (tolerance 1.0

## 선정 결과로부터 경로 찾기

In [21]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from haversine import haversine, Unit

# Function to create distance matrix
def create_distance_matrix(locations):
    distance_matrix = []
    for loc1 in locations:
        row = []
        for loc2 in locations:
            row.append(haversine(loc1, loc2, unit='km'))
        distance_matrix.append(row)
    return distance_matrix

# Create the distance matrix for the selected cafes and warehouse
selected_cafe_locations = [cafe_locations[i] for i in selected_cafes]
all_locations = [selected_warehouse] + selected_cafe_locations  # Starting and ending at the warehouse
distance_matrix = create_distance_matrix(all_locations)

# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix[from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    100000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)

distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Function to print the solution
def print_solution(manager, routing, solution, selected_cafes):
    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    route = []
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route.append(node_index)
        plan_output += ' {} ->'.format(node_index)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(int(previous_index), int(index), 0)
    node_index = manager.IndexToNode(index)
    route.append(node_index)
    plan_output += ' {}\n'.format(node_index)
    print(plan_output)
    print('Route distance: {} km\n'.format(route_distance))
    
    # Map the route to the original indices
    mapped_route = [selected_cafes[i-1] if i > 0 else "Warehouse" for i in route]
    print('Mapped Route:', mapped_route)
    return mapped_route

# Print the solution
if solution:
    mapped_route = print_solution(manager, routing, solution, selected_cafes)
else:
    print('No solution found!')


Objective: 15049
Route:
 0 -> 3 -> 1 -> 4 -> 2 -> 5 -> 7 -> 8 -> 6 -> 9 -> 0

Route distance: 149 km

Mapped Route: ['Warehouse', 5, 1, 9, 3, 11, 19, 21, 17, 23, 'Warehouse']


### 랜덤조건 제거한 창고 및 직영점 위치 선택 코드
(모델 크기 문제로 사용이 불가했습니다. GurobiError: Model too large for size-limited license;)

In [22]:
def read_csv_with_encoding(file_path):
    encodings = ['utf-8', 'ISO-8859-1', 'cp1252']
    for enc in encodings:
        try:
            with open(file_path, newline='', encoding=enc) as csvfile:
                reader = csv.reader(csvfile)
                data = [row for row in reader]
            return data
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to read the file {file_path} with tried encodings.")

# 데이터 저장할 리스트 초기화
cafe_locations = []
sales = []

# CSV 파일에서 좌표 및 매출 읽기
cafe_data = read_csv_with_encoding('각 구청별 좌표.CSV')
for row in cafe_data[0:]:  # Skip header row
    cafe_locations.append((float(row[0]), float(row[1])))
    sales.append(int(row[2]))

# 창고 후보 위치 읽기
warehouse_locations = []
warehouse_data = read_csv_with_encoding('candidate_storages.csv')
for row in warehouse_data[1:]:  # Skip header row
    warehouse_locations.append((float(row[2]), float(row[3])))

# 좌표 개수
num_cafe_coords = len(cafe_locations)
num_warehouse_coords = len(warehouse_locations)

# 거리 행렬 초기화
cafe_distances = [[0.0 for _ in range(num_cafe_coords)] for _ in range(num_cafe_coords)]
warehouse_distances = [[0.0 for _ in range(num_warehouse_coords)] for _ in range(num_cafe_coords)]

# 각 좌표 쌍 간의 거리 계산
for i, loc1 in enumerate(cafe_locations):
    for j, loc2 in enumerate(cafe_locations):
        if i != j:  # 자기 자신과의 거리는 0으로 유지
            cafe_distances[i][j] = haversine(loc1, loc2, unit='km')
        else:
            cafe_distances[i][j] = 1000000000000  # 대규모 상수로 설정하여 자기 자신과의 경로 선택 방지

for i, loc1 in enumerate(cafe_locations):
    for j, loc2 in enumerate(warehouse_locations):
        warehouse_distances[i][j] = haversine(loc1, loc2, unit='km')


# Set
FF = list(range(num_warehouse_coords))  # 모든 창고 후보
CC = list(range(num_cafe_coords))  # 모든 카페 노드

# Parameter
d_fc = {(i, j): cafe_distances[i][j] for i in CC for j in CC}  # 거리 직영점끼리
c_fc = 0.13  # 운임비 창고-직영점
M = 1000000000

# Model
m = gp.Model("coffee_fee")

# Decision Variables
C = m.addVars(CC, CC, vtype=GRB.BINARY, name='C')  # 직영점 끼리의 동선 여부
F = m.addVars(FF, vtype=GRB.BINARY, name='F')      # 평창-창고 여부
Ck = m.addVars(CC, vtype=GRB.BINARY, name='Ck')    # 직영점 여부
V = m.addVars(FF, CC, vtype=GRB.BINARY, name='V')  # 창고-첫 번째 직영점 동선 여부

# Objective function to maximize sales while minimizing costs
alpha = 1e6  # Large constant to prioritize sales

obj = (
    4 * c_fc * gp.quicksum(d_fc[i, j] * C[i, j] for i in CC for j in CC)  # Distribution cost between stores
    - alpha * gp.quicksum(sales[j] * Ck[j] for j in CC)  # Expected sales
)
m.setObjective(obj, GRB.MINIMIZE)

# Constraints
m.addConstr(gp.quicksum(F[i] for i in FF) == 1, "one_warehouse")  # Only one warehouse

# Ensure each store is either selected or not
m.addConstrs(
    (gp.quicksum(C[i, j] for j in CC) <= Ck[i] for i in CC), name="path_constraint"
)
m.addConstrs(
    (gp.quicksum(C[i, j] for i in CC) <= Ck[j] for j in CC), name="path_constraint"
)

# Ensure adjacency constraint for stores
adjacency_constraints = [
    (0, 1), (0, 3), (0, 4), (1, 2), (1, 4), (2, 4), (2, 5), (2, 10), (2, 4), (3, 4),
    (3, 6), (3, 7), (3, 8), (4, 9), (5, 10), (6, 7), (6, 11), (7, 11), (7, 12),
    (8, 9), (8, 12), (8, 13), (8, 17), (8, 18), (9, 10), (9, 13), (10, 13), (11, 12),
    (11, 15), (12, 17), (13, 17), (13, 18), (14, 15), (14, 19), (14, 20), (15, 16),
    (15, 20), (15, 21), (16, 17), (16, 21), (17, 18), (17, 22), (18, 22), (18, 23),
    (19, 20), (20, 21), (21, 22), (22, 23), (22, 24), (23, 24)
]

m.addConstrs(
    (Ck[i] + Ck[j] <= 1 for i, j in adjacency_constraints if i in CC and j in CC), name="adjacency_constraint"
)

# Constraint on total sales
m.addConstr(
    gp.quicksum(0.044 * sales[j] * Ck[j] for j in CC) <= 1388431.8, name="sales_constraint"
)

# Solve TSP including the warehouse
for i in FF:
    for j in CC:
        if j != i:
            m.addConstr(V[i, j] <= F[i], name=f"first_delivery_{i}_{j}")

m.optimize()

# Extract selected cafes
selected_cafes = []
if m.status == GRB.Status.OPTIMAL:
    print('Optimal objective: %g' % m.objVal)
    for v in m.getVars():
        if v.x > 0:
            print('%s: %g' % (v.varName, v.x))
    
    # Extract selected cafes
    selected_cafes = [j for j in CC if Ck[j].x > 0.5]
else:
    print('No optimal solution found')

# Print selected cafes
print("Selected Cafes:", selected_cafes)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information